In [2]:
import os
import glob
import pydicom
import nibabel as nib
import pandas as pd
import numpy as np
from pydicom.pixel_data_handlers.util import apply_voi_lut
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import cv2

from tqdm import tqdm

In [3]:
DATA_DIR = "/Volumes/SSD970/"
TRAIN_IMAGE_DIR = os.path.join(DATA_DIR, "train_images")

In [4]:
def read_patient_z(patient_id):
    patient_dir = os.path.join(TRAIN_IMAGE_DIR, patient_id)
    num_slices = len(glob.glob(patient_dir + f"/*"))
    
    z_positions = np.zeros(num_slices)
    pass_count = 1
    for i in range(num_slices):
        # dicom filenames start from 1
        dcm_path = os.path.join(patient_dir, f"{i+pass_count}.dcm")
        while os.path.exists(dcm_path) is False:
            print(f"pass {dcm_path}")
            pass_count += 1
            dcm_path = os.path.join(patient_dir, f"{i+pass_count}.dcm")
        ds = pydicom.dcmread(dcm_path)
        z_positions[i] = ds.ImagePositionPatient[2]
        
    return z_positions


In [14]:
patient_id = "1.2.826.0.1.3680043.29630"
z_positions = read_patient_z(patient_id)
z_spacing = z_positions[1:] - z_positions[:-1]
mean_spacing = np.round(np.mean(z_spacing), decimals=4)
first_spacing = np.round(z_spacing[0], decimals=4)
if mean_spacing != first_spacing:
    print(f'not equal {mean_spacing} != {first_spacing} at {patient_id}')
else:
    print('is equal')
z_spacing

pass /Volumes/SSD970/train_images/1.2.826.0.1.3680043.29630/1.dcm
not equal -0.9703 != -0.963 at 1.2.826.0.1.3680043.29630


array([-0.96300147, -0.961998  , -0.963002  , -0.96199762, -0.962     ,
       -0.9630009 , -0.96200148, -0.962     , -0.963     , -0.962     ,
       -0.963     , -0.96199853, -0.96200147, -0.963     , -0.962     ,
       -0.962     , -0.963     , -0.962     , -0.963     , -0.96199853,
       -0.96200147, -0.963     , -0.962     , -0.96200051, -0.96299962,
       -0.96200039, -0.96299988, -0.96199961, -0.96200012, -0.9630004 ,
       -0.96200014, -0.96200109, -0.963     , -0.96199704, -0.96300149,
       -0.99692979, -0.9988397 , -0.99883914, -0.99784001, -0.99883745,
       -0.99883633, -0.9978352 , -0.99883551, -0.99883408, -0.99883187,
       -0.99783396, -0.99883022, -0.99883078, -0.99783055, -0.99882822,
       -0.99882773, -0.99882865, -0.99782336, -0.99882549, -0.99882401,
       -0.96849312, -0.96399962, -0.96300038, -0.96399962, -0.96300146,
       -1.13496736, -1.16919285, -1.167797  , -1.16740057, -1.16799504,
       -1.16759173, -1.16718357, -1.16578086, -1.1663663 , -1.16

array([-0.625, -0.625, -0.625, -0.625, -0.625, -0.625, -0.625, -0.625,
       -0.625, -0.625, -0.625, -0.625, -0.625, -0.625, -0.625, -0.625,
       -0.625, -0.625, -0.625, -0.625, -0.625, -0.625, -0.625, -0.625,
       -0.625, -0.625, -0.625, -0.625, -0.625, -0.625, -0.625, -0.625,
       -0.625, -0.625, -0.625, -0.625, -0.625, -0.625, -0.625, -0.625,
       -0.625, -0.625, -0.625, -0.625, -0.625, -0.625, -0.625, -0.625,
       -0.625, -0.625, -0.625, -0.625, -0.625, -0.625, -0.625, -0.625,
       -0.625, -0.625, -0.625, -0.625, -0.625, -0.625, -0.625, -0.625,
       -0.625, -0.625, -0.625, -0.625, -0.625, -0.625, -0.625, -0.625,
       -0.625, -0.625, -0.625, -0.625, -0.625, -0.625, -0.625, -0.625,
       -0.625, -0.625, -0.625, -0.625, -0.625, -0.625, -0.625, -0.625,
       -0.625, -0.625, -0.625, -0.625, -0.625, -0.625, -0.625, -0.625,
       -0.625, -0.625, -0.625, -0.625, -0.625, -0.625, -0.625, -0.625,
       -0.625, -0.625, -0.625, -0.625, -0.625, -0.625, -0.625, -0.625,
      

In [12]:
patient_paths = glob.glob(TRAIN_IMAGE_DIR + f"/**")
for path in tqdm(patient_paths):
    patient_id = path.split("/")[-1]
    
    z_positions = read_patient_z(patient_id)
    z_spacing = z_positions[1:] - z_positions[:-1]
    mean_spacing = np.round(np.mean(z_spacing), decimals=4)
    first_spacing = np.round(z_spacing[0], decimals=4)
    if mean_spacing != first_spacing:
        print(f'not equal {mean_spacing} != {first_spacing} at {patient_id}')

  0%|          | 4/2019 [00:03<29:08,  1.15it/s]

not equal -0.3125 != -0.313 at 1.2.826.0.1.3680043.10016


  0%|          | 10/2019 [00:07<25:27,  1.32it/s]


KeyboardInterrupt: 

glob.glob 를 쓰면 파일 순서가 개판된다는거
그래서 range(num_slices) 를 썼다. 

In [ ]:
glob.glob(TRAIN_IMAGE_DIR + f"/1.2.826.0.1.3680043.29872/*")

In [ ]:
os.path.exists(TRAIN_IMAGE_DIR + f"/1.2.826.0.1.3680043.29872/2.dcm")